In [2]:
!pip install google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 54.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Uninstalling protobuf-3.19.4:
      Successfully uninstalled protobuf-3.19.4
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.9.0
    Uninstalling google-auth-2.9.0:
      Successfully uninstalled google-auth-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tens

In [3]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./documents-processing-demo-2c53a186b422.json"


In [ ]:
from google.cloud import storage
import os
import random

def create_small_dataset_gcp(src_bucket_name, src_folder, dst_bucket_name, train_ratio=0.8, images_per_category=200):
    storage_client = storage.Client()
    src_bucket = storage_client.get_bucket(src_bucket_name)
    dst_bucket = storage_client.get_bucket(dst_bucket_name)

    for phase in ['train', 'val']:
        for category in [blob.name.split("/")[2] for blob in src_bucket.list_blobs(prefix=f'{src_folder}/train/') if blob.name.endswith('/')]:
            print(f"Processing {phase}/{category}...")
            src_category_prefix = f'{src_folder}/{phase}/{category}/'
            dst_category_prefix = f'{dst_root}/{phase}/{category}/'

            image_blobs = [blob for blob in src_bucket.list_blobs(prefix=src_category_prefix) if not blob.name.endswith('/')]
            random.shuffle(image_blobs)

            if phase == 'train':
                num_train_images = int(images_per_category * train_ratio)
                selected_blobs = image_blobs[:num_train_images]
            else:  # phase == 'val'
                num_val_images = images_per_category - int(images_per_category * train_ratio)
                selected_blobs = image_blobs[-num_val_images:]

            for blob in selected_blobs:
                dst_blob_name = dst_category_prefix + os.path.basename(blob.name)
                dst_blob = dst_bucket.blob(dst_blob_name)
                print(f"Copying {blob.name} to {dst_blob_name}...")
                dst_bucket.copy_blob(blob, dst_bucket, dst_blob_name)

def delete_existing_small_dataset_gcp(dst_bucket_name):
    storage_client = storage.Client()
    dst_bucket = storage_client.get_bucket(dst_bucket_name)

    for phase in ['train', 'val']:
        phase_prefix = f'{dst_root}/{phase}/'
        blobs = list(dst_bucket.list_blobs(prefix=phase_prefix))
        for blob in blobs:
            print(f"Deleting {blob.name}...")
            blob.delete()

# Update these variables with your GCP bucket names and source folder
src_bucket_name = 'rvl-cdip-full'
src_folder = 'rvl-cdip-small'
dst_bucket_name = 'rvl-cdip-small-200'

# Uncomment the following line to delete the existing smaller dataset before creating a new one
# delete_existing_small_dataset_gcp(dst_bucket_name)

create_small_dataset_gcp(src_bucket_name, src_folder, dst_bucket_name, train_ratio=0.8, images_per_category=200)
